In [29]:
import utils
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
import joblib
from skrub import TableVectorizer
import optuna
from sklearn.model_selection import cross_val_score

In [30]:
X, y, X_test = utils.get_and_process_data()

/Users/gregzguegue/Python_Data_Challenge/utils.py:216: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  (non_nan_values - value).abs().argmin()
/Users/gregzguegue/Python_Data_Challenge/utils.py:216: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  (non_nan_values - value).abs().argmin()
/Users/gregzguegue/Python_Data_Challenge/utils.py:382: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are

In [45]:
def create_pipeline_TV(df, model=None):
    # Best params XGBoost from optuna
    best_params_XGBoost = {
    "n_estimators": 190,
    "max_depth": 6,
    "learning_rate": 0.11981406065938047,
    "subsample": 0.6544353679306312,
    "colsample_bytree": 0.6106475983764972,
    "random_state": 42,
}
    # Define TableVectorizer for preprocessing
    preprocessor = ColumnTransformer(
        transformers=[
            (
                "vectorizer",
                TableVectorizer(),
                df.columns,
            )  # Apply TableVectorizer to all columns
        ]
    )

    # Use the provided model or initialize a default XGBRegressor
    if model is None:
        model = XGBRegressor(**best_params_XGBoost)

    # Create the pipeline
    pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])

    return pipeline

In [27]:
# Define the Optuna objective function for minimizing RMSE
def objective(trial, X, y):
    # Sample hyperparameters
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    subsample = trial.suggest_float("subsample", 0.5, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)

    # Create model with sampled hyperparameters
    model = XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=42,
    )

    # Create the pipeline
    pipeline = create_pipeline_TV(df=X, model=model)

    # Evaluate using cross-validation
    scores = cross_val_score(pipeline, X, y, cv=5, scoring="neg_mean_squared_error")

    # Convert negative MSE to RMSE
    rmse = (-scores.mean()) ** 0.5
    return rmse

# Run the Optuna study
def run_optuna(X, y):
    study = optuna.create_study(direction="minimize")  # Minimize RMSE
    study.optimize(lambda trial: objective(trial, X, y), n_trials=50)
    return study

In [28]:
study = run_optuna(X, y)

[I 2024-12-11 15:26:54,052] A new study created in memory with name: no-name-f10783f7-c8c2-4705-91fa-87e59c9520d7
/opt/anaconda3/envs/py312-myenv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/envs/py312-myenv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/envs/py312-myenv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/envs/py312-myenv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] dur

KeyboardInterrupt: 

In [ ]:
# Best parameters and score
print("Best Parameters:", study.best_params)
print("Best Score:", study.best_value)

In [ ]:
best_params_XGBoost = {
    "n_estimators": 190,
    "max_depth": 6,
    "learning_rate": 0.11981406065938047,
    "subsample": 0.6544353679306312,
    "colsample_bytree": 0.6106475983764972,
}

In [46]:
pipeline = create_pipeline_TV(X)

In [47]:
pipeline.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('vectorizer',
                                                  TableVectorizer(),
                                                  Index(['counter_id', 'site_id', 'date', 'counter_installation_date',
       'counter_technical_id', 'latitude', 'longitude',
       'Pressure_Tendency_(hPa/3h)', 'Pressure_Tendency_Code',
       'Wind_Direction_(°)', 'Wind_Speed_(m/s)', 'Air_Temperature_(°C)',
       'Dew_Point_Temperature_(°C...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.11981406065938047, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=6, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=190, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [48]:
submission = utils.test_fit_and_submission(X_test, pipeline)